<a href="https://colab.research.google.com/github/jakkaphat1/Project1_doc/blob/main/(Text_or_Image_Only)_Assignment_Plagiarism_Detection_V5_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Document**
Text Embedding Model : LaBse

Lexical Matching Process : N-gram + Jaccard Similarity

Template Removing

------------------------
Image Embedding Model : clip-vit-large-patch14

Embedding similarity choosen : Matching Count (ใช้จำนวนภาพที่มี Similarity > threshold ที่ตั้งไว้)

------------------------
- เพิ่มวิธีการคัดรูปภาพวาดออกมา



# Requirements

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

import os
from slugify import slugify

import shutil


In [2]:
!pip install -q pythainlp
from pythainlp.tokenize import word_tokenize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 46.5 MB/s eta 0:00:00


In [3]:
!pip install pdf2image
from pdf2image import convert_from_path


In [4]:
!pip install pymupdf
!pip install pytesseract
!pip install pillow pdf2image
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!apt-get install tesseract-ocr-tha

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 13.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.10 [186 kB]
Fetched 186 kB in 0s (386 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126371 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.10_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.10) ...
Setting up poppler-utils (22.02.0-2ubuntu0.10) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract

In [5]:
!pip install pinecone-client transformers pymupdf


In [6]:
pip install python-slugify

In [7]:
pip install sentence-transformers


In [8]:
pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [9]:
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path
from PIL import Image

pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

In [10]:
from google.colab import files

In [11]:
import tempfile

# Embedding Models

In [12]:
import fitz  # PyMuPDF
import tempfile
import os
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
from sentence_transformers import SentenceTransformer

# โหลดโมเดล Embedding
labse_model = SentenceTransformer("sentence-transformers/LaBSE")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

# Pinecone

In [13]:
from pinecone import Pinecone, ServerlessSpec
# Pinecone API Key และ Environment (Region)
PINECONE_API_KEY = "pcsk_3q7EHL_2jLmXAfQ9rXqj3h2tHaG1YjjYuQ5HnEtNHucWasCCsxdvr8qvLHqPip4Aa17ziJ"
PINECONE_ENVIRONMENT = "us-east-1"  # Region

# สร้าง Pinecone Client Instance
pc = Pinecone(api_key=PINECONE_API_KEY)

# ตรวจสอบ Index ก่อนสร้าง
if "text-index" in pc.list_indexes().names():
    print(pc.list_indexes().names())

if "image-index" in pc.list_indexes().names():
    print(pc.list_indexes().names())

# โหลด Index
text_index = pc.Index("text-index")
image_index = pc.Index("image-index")



['text-1024', 'text-index', 'image-1024', 'image-index']
['text-1024', 'text-index', 'image-1024', 'image-index']


In [14]:
print(text_index.describe_index_stats())

{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


In [15]:
print(image_index.describe_index_stats())

{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 10}},
 'total_vector_count': 10,
 'vector_type': 'dense'}


# Plagiarism Process

In [16]:
from google.colab import files

def upload_pdfs():
    uploaded = files.upload()
    pdf_paths = list(uploaded.keys())
    return pdf_paths

# เลือกโหมดการประมวลผล
print("เลือกโหมดการประมวลผล PDF:")
print("1 = ประมวลผลเฉพาะข้อความ (Text Only)")
print("2 = ประมวลผลเฉพาะรูปภาพ (Image Only)")

try:
    PROCESSING_MODE = int(input("เลือกโหมด 1 หรือ 2: "))
    if PROCESSING_MODE not in [1, 2]:
        raise ValueError("กรุณาเลือกเฉพาะ 1 หรือ 2")
except:
    print("กรุณาใส่ค่าที่ถูกต้อง (1 หรือ 2)")
    PROCESSING_MODE = 1

print(f"โหมดที่เลือก: {PROCESSING_MODE}")

pdf_paths = upload_pdfs()

เลือกโหมดการประมวลผล PDF:
1 = ประมวลผลเฉพาะข้อความ (Text Only)
2 = ประมวลผลเฉพาะรูปภาพ (Image Only)
เลือกโหมด 1 หรือ 2: 2
โหมดที่เลือก: 2


Saving 663380118-0.pdf to 663380118-0.pdf
Saving 663380129-5.pdf to 663380129-5.pdf
Saving 663380138-4.pdf to 663380138-4.pdf
Saving 663380445-5.pdf to 663380445-5.pdf
Saving 663380451-0.pdf to 663380451-0.pdf
Saving 663380456-0.pdf to 663380456-0.pdf


In [17]:
def l2norm(x: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(x)
    return x / (n + 1e-12)


In [18]:
def extract_text_and_embed(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc]).strip()

    if not text:
        # ใช้ OCR กรณีที่ PyMuPDF ไม่เห็นข้อความ
        images = convert_from_path(pdf_path)
        ocr_text = []
        for img in images:
            ocr_text.append(pytesseract.image_to_string(img, lang="tha+eng"))
        text = "\n".join(ocr_text).strip()

    # ตัดคำด้วย PyThaiNLP
    tokens = word_tokenize(text, engine="newmm")
    tokenized_text = " ".join(tokens)

    text_embedding = labse_model.encode(tokenized_text)
    if isinstance(text_embedding, list):
        text_embedding = np.array(text_embedding, dtype=np.float32)
    else:
        text_embedding = np.asarray(text_embedding, dtype=np.float32)
    text_embedding = l2norm(text_embedding)

    return tokenized_text, text_embedding



def extract_images_and_embed(pdf_path):
    doc = fitz.open(pdf_path)
    image_embeddings = []
    temp_dir = tempfile.mkdtemp()

    for page_num in range(len(doc)):
        embedded_found = False
        for img_index, img_info in enumerate(doc[page_num].get_images(full=True)):
            xref = img_info[0]
            base_image = doc.extract_image(xref)
            img_bytes = base_image["image"]
            img_path = os.path.join(temp_dir, f"page_{page_num}_img_{img_index}.png")
            with open(img_path, "wb") as f:
                f.write(img_bytes)
            img = Image.open(img_path).convert("RGB")

            inputs = clip_processor(images=img, return_tensors="pt")
            with torch.no_grad():
                img_embedding = clip_model.get_image_features(**inputs).squeeze()
            if hasattr(img_embedding, "cpu"):
                img_embedding = img_embedding.cpu()
            img_embedding = np.asarray(img_embedding, dtype=np.float32)
            img_embedding = l2norm(img_embedding)
            image_embeddings.append(img_embedding)
            embedded_found = True

        if not embedded_found:
            mat = fitz.Matrix(3.0, 3.0)  # zoom 3x
            pix = doc[page_num].get_pixmap(matrix=mat)
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

            inputs = clip_processor(images=img, return_tensors="pt")
            with torch.no_grad():
                img_embedding = clip_model.get_image_features(**inputs).squeeze()

            if hasattr(img_embedding, "cpu"):
                img_embedding = img_embedding.cpu()
            img_embedding = np.asarray(img_embedding, dtype=np.float32)
            img_embedding = l2norm(img_embedding)
            image_embeddings.append(img_embedding)

    shutil.rmtree(temp_dir)
    return image_embeddings

In [19]:
from difflib import SequenceMatcher

def remove_template_text(student_text, template_text, threshold=0.9, return_removed=False):
    student_lines = student_text.strip().split('\n')
    template_lines = template_text.strip().split('\n')

    result = []
    removed = []

    for s_line in student_lines:
        match_found = any(
            SequenceMatcher(None, s_line.strip(), t_line.strip()).ratio() > threshold
            for t_line in template_lines)
        if not match_found:
            result.append(s_line)
        else:
            removed.append(s_line)

    if return_removed:
        return "\n".join(result).strip(), "\n".join(removed).strip()
    else:
        return "\n".join(result).strip()

In [20]:
def to_ascii_id(filename):
    return slugify(filename, separator="_")

In [21]:
def upload_text_embedding(doc_id, text_embedding):
    if text_embedding is not None and len(text_embedding) > 0:
        text_index.upsert([(f"text_{doc_id}", text_embedding.tolist(), {"type": "text"})])

def upload_image_embedding(doc_id, image_embeddings):
    if image_embeddings:
        for i, img_emb in enumerate(image_embeddings):
            image_index.upsert([(f"image_{doc_id}_{i}", img_emb.tolist(), {"type": "image"})])

In [23]:
USE_TEMPLATE = 0
template_text_raw = ""

if PROCESSING_MODE == 1:
    print("\nต้องการใช้ไฟล์ Template ของอาจารย์เพื่อลบข้อความซ้ำหรือไม่?")
    print("0 = ไม่ใช้ template")
    print("1 = ใช้ template")
    try:
        USE_TEMPLATE = int(input("เลือกโหมด 0 หรือ 1 : "))
    except:
        USE_TEMPLATE = 0

    if USE_TEMPLATE == 1:
        print("โปรดอัปโหลดไฟล์ Template PDF ของอาจารย์")
        uploaded_tpl = files.upload()
        template_path = list(uploaded_tpl.keys())[0]
        template_text_raw, _ = extract_text_and_embed(template_path)
    elif USE_TEMPLATE != 0:
        raise ValueError("ค่า USE_TEMPLATE ต้องเป็น 0 หรือ 1 เท่านั้น")


In [24]:
raw_texts = {}

for pdf_path in pdf_paths:
    doc_id = to_ascii_id(os.path.basename(pdf_path).split('.')[0])
    print(f"\n--- กำลังประมวลผล: {doc_id} ---")

    # โหมด 1: ประมวลผลเฉพาะข้อความ
    if PROCESSING_MODE == 1:
        print("โหมด: ประมวลผลเฉพาะข้อความ")

        # ดึงข้อความต้นฉบับของนักศึกษา
        student_text_raw, _ = extract_text_and_embed(pdf_path)

        if USE_TEMPLATE == 0:
            clean_text = student_text_raw
            removed_text = ""
        elif USE_TEMPLATE == 1:
            clean_text, removed_text = remove_template_text(
                student_text_raw,
                template_text_raw,
                return_removed=True
            )
            print(f"ข้อความที่ถูกลบออกจาก Template: {len(removed_text)} ตัวอักษร")

        # ตัดคำ + ฝัง embedding
        # tokens = word_tokenize(clean_text, engine="newmm")
        # tokenized_text = " ".join(tokens)
        # text_embedding = labse_model.encode(tokenized_text)

        # # อัปโหลดเข้า Vector DB (ข้อความ)
        # upload_text_embedding(doc_id, text_embedding)

        tokens = word_tokenize(clean_text, engine="newmm")
        tokenized_text = " ".join(tokens)
        text_embedding = labse_model.encode(tokenized_text)
        text_embedding = np.asarray(text_embedding, dtype=np.float32)
        text_embedding = l2norm(text_embedding)
        upload_text_embedding(doc_id, text_embedding)
        # เก็บข้อความสะอาดไว้ใช้เปรียบเทียบ
        raw_texts[doc_id] = clean_text

        print("ประมวลผลข้อความเสร็จสิ้น")

    # โหมด 2: ประมวลผลเฉพาะรูปภาพ
    elif PROCESSING_MODE == 2:
        print("โหมด: ประมวลผลเฉพาะรูปภาพ")

        # ฝัง image embedding และอัปโหลด
        image_embeddings = extract_images_and_embed(pdf_path)
        upload_image_embedding(doc_id, image_embeddings)

        # เก็บข้อความเปล่าไว้
        raw_texts[doc_id] = ""

        print(f"ประมวลผลรูปภาพเสร็จสิ้น ({len(image_embeddings)} รูป)")

print("\n ประมวลผลไฟล์ทั้งหมดเสร็จสิ้น")


--- กำลังประมวลผล: 663380118_0 ---
โหมด: ประมวลผลเฉพาะรูปภาพ
ประมวลผลรูปภาพเสร็จสิ้น (1 รูป)

--- กำลังประมวลผล: 663380129_5 ---
โหมด: ประมวลผลเฉพาะรูปภาพ
ประมวลผลรูปภาพเสร็จสิ้น (1 รูป)

--- กำลังประมวลผล: 663380138_4 ---
โหมด: ประมวลผลเฉพาะรูปภาพ
ประมวลผลรูปภาพเสร็จสิ้น (1 รูป)

--- กำลังประมวลผล: 663380445_5 ---
โหมด: ประมวลผลเฉพาะรูปภาพ
ประมวลผลรูปภาพเสร็จสิ้น (1 รูป)

--- กำลังประมวลผล: 663380451_0 ---
โหมด: ประมวลผลเฉพาะรูปภาพ
ประมวลผลรูปภาพเสร็จสิ้น (1 รูป)

--- กำลังประมวลผล: 663380456_0 ---
โหมด: ประมวลผลเฉพาะรูปภาพ
ประมวลผลรูปภาพเสร็จสิ้น (1 รูป)

 ประมวลผลไฟล์ทั้งหมดเสร็จสิ้น


In [25]:
def preprocess_text(text):
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    tokens = word_tokenize(text, engine="newmm")
    return " ".join(tokens)

In [26]:
def compute_lexical_similarity(text1, text2, ngram_range=(1,3)):
    t1, t2 = preprocess_text(text1), preprocess_text(text2)
    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=ngram_range)
    tfidf = vectorizer.fit_transform([t1, t2])
    return cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]

def compute_hybrid_similarity(text1, text2, emb1, emb2, alpha=0.5, beta=0.5):
    semantic = cosine_similarity([emb1], [emb2])[0][0]
    lexical = compute_lexical_similarity(text1, text2)
    hybrid = alpha * semantic + beta * lexical
    return hybrid, semantic, lexical

In [27]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def compute_average_image_similarity(image_embeddings, doc_id_1, doc_id_2):
    """
    คำนวณค่าเฉลี่ยความคล้ายคลึงของรูปภาพทั้งหมดระหว่าง 2 เอกสาร
    """
    similarities = []
    keys1 = [k for k in image_embeddings if k.startswith(f"image_{doc_id_1}_")]
    keys2 = [k for k in image_embeddings if k.startswith(f"image_{doc_id_2}_")]

    # หากเอกสารใดเอกสารหนึ่งไม่มีรูปภาพ ให้คืนค่า 0.0
    if not keys1 or not keys2:
        return 0.0

    # เปรียบเทียบรูปภาพทุกคู่ที่เป็นไปได้
    for k1 in keys1:
        for k2 in keys2:
            sim = cosine_similarity([image_embeddings[k1]], [image_embeddings[k2]])[0][0]
            similarities.append(sim)

    # คำนวณค่าเฉลี่ยจากรายการ similarities ทั้งหมด
    # หากไม่มีค่าใดๆ เลย (กรณีฉุกเฉิน) ให้คืนค่า 0.0
    average_sim = np.mean(similarities) if similarities else 0.0

    return average_sim

In [28]:
def get_all_text_embeddings():
    stats = text_index.describe_index_stats()
    total_vectors = stats["total_vector_count"]
    if total_vectors == 0:
        return {}
    text_vectors = text_index.query(vector=[0]*labse_model.get_sentence_embedding_dimension(), top_k=total_vectors, include_values=True)
    return {match["id"]: np.array(match["values"]) for match in text_vectors["matches"]}

def get_all_image_embeddings():
    stats = image_index.describe_index_stats()
    total_vectors = stats["total_vector_count"]

    if total_vectors == 0:
        print("ไม่มีรูปภาพใน index")
        return {}

    #ใช้ query() เพื่อดึงค่าเวกเตอร์ทั้งหมดแทน fetch()
    image_vectors = image_index.query(
        vector=[0] * 768,
        top_k=total_vectors,
        include_values=True
    )

    return {match["id"]: np.array(match["values"]) for match in image_vectors["matches"]}

In [29]:
def compare_all_documents():
    text_embeddings = get_all_text_embeddings()
    image_embeddings = get_all_image_embeddings()
    results = []

    IMAGE_SIMILARITY_THRESHOLD = 0.80

    doc_ids = list(raw_texts.keys())

    print(f"\n--- เปรียบเทียบเอกสาร (โหมด {PROCESSING_MODE}) ---")

    for i in range(len(doc_ids)):
        for j in range(i + 1, len(doc_ids)):
            id1, id2 = doc_ids[i], doc_ids[j]

            # ตรวจสอบโหมดการประมวลผล
            if PROCESSING_MODE == 1:  # เฉพาะข้อความ
                t1, t2 = raw_texts[id1], raw_texts[id2]
                e1, e2 = text_embeddings[f"text_{id1}"], text_embeddings[f"text_{id2}"]
                hybrid, semantic, lexical = compute_hybrid_similarity(t1, t2, e1, e2)

                if hybrid >= 0.90:
                    level = "เหมือนมาก"
                elif hybrid >= 0.75:
                    level = "คล้ายสูง"
                elif hybrid >= 0.60:
                    level = "คล้ายระดับกลาง"
                else:
                    level = "ไม่คล้าย"

                level_message_map = {
                    "เหมือนมาก": "ข้อความมีความเหมือนสูงมาก",
                    "คล้ายสูง": "ข้อความมีความคล้ายสูง",
                    "คล้ายระดับกลาง": "ข้อความมีความคล้ายระดับกลาง",
                    "ไม่คล้าย": "ข้อความมีความคล้ายน้อย",
                }
                text_message = level_message_map[level]

                print("===============================================================================")
                print(f"{id1} vs {id2}")
                print(f"   Semantic: {semantic:.4f} | Lexical: {lexical:.4f} | Hybrid: {hybrid:.4f}")
                print(f"   Text Message: {text_message}")

                results.append({
                    "doc_1": id1,
                    "doc_2": id2,
                    "hybrid_similarity": hybrid,
                    "semantic_similarity": semantic,
                    "lexical_similarity": lexical,
                    "image_similarity": 0.0,
                    "level": level,
                    "text_message": text_message,
                    "matched_images": 0,
                    "total_images": 0
                })

            elif PROCESSING_MODE == 2:  # เฉพาะรูปภาพ
                image_sim = compute_average_image_similarity(image_embeddings, id1, id2)
                matched_images = 0
                keys1 = [k for k in image_embeddings if k.startswith(f"image_{id1}_")]
                keys2 = [k for k in image_embeddings if k.startswith(f"image_{id2}_")]
                total_images = len(keys1) * len(keys2)

                if total_images > 0:
                    for k1 in keys1:
                        for k2 in keys2:
                            sim = cosine_similarity([image_embeddings[k1]], [image_embeddings[k2]])[0][0]
                            if sim >= IMAGE_SIMILARITY_THRESHOLD:
                                matched_images += 1

                level = "เหมือนมาก" if image_sim >= 0.9 else \
                        "คล้ายสูง" if image_sim >= 0.75 else \
                        "คล้ายระดับกลาง" if image_sim >= 0.6 else "ไม่คล้าย"

                print("===============================================================================")
                print(f"{id1} vs {id2}")
                print(f"   Image Similarity: {image_sim:.4f}")
                print(f"   Image: ภาพเหมือน {matched_images} จาก {total_images} รูป")

                results.append({
                    "doc_1": id1,
                    "doc_2": id2,
                    # "hybrid_similarity": image_sim,
                    "semantic_similarity": 0.0,
                    "lexical_similarity": 0.0,
                    "image_similarity": image_sim,
                    "level": level,
                    "text_message": "ไม่มีการประมวลผลข้อความ",
                    "matched_images": matched_images,
                    "total_images": total_images
                })

    return results

In [30]:
compare_results = compare_all_documents()


--- เปรียบเทียบเอกสาร (โหมด 2) ---
663380118_0 vs 663380129_5
   Image Similarity: 0.7943
   Image: ภาพเหมือน 0 จาก 1 รูป
663380118_0 vs 663380138_4
   Image Similarity: 0.7217
   Image: ภาพเหมือน 0 จาก 1 รูป
663380118_0 vs 663380445_5
   Image Similarity: 0.9346
   Image: ภาพเหมือน 1 จาก 1 รูป
663380118_0 vs 663380451_0
   Image Similarity: 0.6803
   Image: ภาพเหมือน 0 จาก 1 รูป
663380118_0 vs 663380456_0
   Image Similarity: 0.9300
   Image: ภาพเหมือน 1 จาก 1 รูป
663380129_5 vs 663380138_4
   Image Similarity: 0.8586
   Image: ภาพเหมือน 1 จาก 1 รูป
663380129_5 vs 663380445_5
   Image Similarity: 0.7652
   Image: ภาพเหมือน 0 จาก 1 รูป
663380129_5 vs 663380451_0
   Image Similarity: 0.8161
   Image: ภาพเหมือน 1 จาก 1 รูป
663380129_5 vs 663380456_0
   Image Similarity: 0.7870
   Image: ภาพเหมือน 0 จาก 1 รูป
663380138_4 vs 663380445_5
   Image Similarity: 0.7340
   Image: ภาพเหมือน 0 จาก 1 รูป
663380138_4 vs 663380451_0
   Image Similarity: 0.8066
   Image: ภาพเหมือน 1 จาก 1 รูป
6633801

In [31]:
import pandas as pd
from ipywidgets import interact, Dropdown, IntSlider, Checkbox

# ===== Helper: เลือก metric และข้อความให้ถูกกับโหมด =====
def _extract_fields_for_mode(r, mode: int, round_ndigits: int):
    if mode == 1:  # ข้อความ
        sim = float(r.get('hybrid_similarity', 0.0))
        semantic = float(r.get('semantic_similarity', 0.0))
        lexical = float(r.get('lexical_similarity', 0.0))
        # ใช้ข้อความที่คำนวณไว้แล้ว หากไม่มีให้สร้างตาม threshold
        msg = r.get('text_message')
        if not msg:
            if sim >= 0.90:
                msg = "ข้อความมีความเหมือนสูงมาก"
            elif sim >= 0.75:
                msg = "ข้อความมีความคล้ายสูง"
            elif sim >= 0.60:
                msg = "ข้อความมีความคล้ายระดับกลาง"
            else:
                msg = "ข้อความมีความคล้ายน้อย"
        # ประโยคสรุปแบบสั้นเมื่อพิมพ์
        pretty_line = f"Hybrid={sim:.{round_ndigits}f} | Semantic={semantic:.{round_ndigits}f} | Lexical={lexical:.{round_ndigits}f} | {msg}"
        extra_cols = {
            'semantic': semantic,
            'lexical': lexical,
            'message': msg,
            'pretty_line': pretty_line
        }
        return sim, extra_cols

    else:  # โหมดรูปภาพ (mode == 2) หรืออื่น ๆ ให้ถือเป็นรูปภาพ
        sim = float(r.get('image_similarity', 0.0))
        matched = int(r.get('matched_images', 0) or 0)
        total = int(r.get('total_images', 0) or 0)
        if total > 0:
            msg = f"ภาพเหมือน {matched} จาก {total} รูป"
        else:
            msg = "ไม่มีภาพให้เปรียบเทียบ"
        pretty_line = f"ImageSim={sim:.{round_ndigits}f} | {msg}"
        extra_cols = {
            'matched_images': matched,
            'total_images': total,
            'message': msg,
            'pretty_line': pretty_line
        }
        return sim, extra_cols


# ===== ฟังก์ชันหลัก: เทียบ seed กับทุกไฟล์ =====
def list_all_vs_seed(seed_id: str, round_ndigits: int = 3, sort_desc: bool = True,
                     also_print: bool = True, mode: int = None):
    """
    seed_id: ไฟล์ตั้งต้น (doc_1)
    mode: ถ้า None จะอ่านจากตัวแปร PROCESSING_MODE (1=ข้อความ, 2=รูปภาพ)
    """
    # เดาโหมดจากตัวแปร global ถ้าไม่ได้ส่งมา
    try:
        _mode = PROCESSING_MODE if mode is None else mode
    except NameError:
        _mode = 2  # ถ้าไม่มี PROCESSING_MODE ก็ถือว่าโหมดรูปภาพ

    rows = []
    for r in compare_results:   # compare_results = compare_all_documents()
        if r['doc_1'] == seed_id or r['doc_2'] == seed_id:
            other = r['doc_2'] if r['doc_1'] == seed_id else r['doc_1']

            sim, extra = _extract_fields_for_mode(r, _mode, round_ndigits)

            row = {
                'doc_1': seed_id,
                'doc_2': other,
                'similarity': sim,
                'similarity_rounded': round(sim, round_ndigits),
                'message': extra['message'],
                'pretty': extra['pretty_line'],
            }
            # แนบคอลัมน์พิเศษตามโหมด
            if _mode == 1:
                row['semantic'] = extra['semantic']
                row['lexical']  = extra['lexical']
            else:
                row['matched_images'] = extra.get('matched_images', 0)
                row['total_images']   = extra.get('total_images', 0)

            rows.append(row)

    out = pd.DataFrame(rows)
    if out.empty:
        print(f"ไม่พบคู่ของ seed: {seed_id}")
        return out

    out = out.sort_values('similarity', ascending=not sort_desc).reset_index(drop=True)

    if also_print:
        print(f"\nไฟล์ {seed_id}")
        for _, r in out.iterrows():
            print(f"เทียบกับ {r['doc_2']} มีค่า similarity = {r['similarity_rounded']:.{round_ndigits}f} | {r['message']}")

    return out


# ===== Widget UI =====
# สร้างรายการเอกสารจาก compare_results
docs = sorted(set([r['doc_1'] for r in compare_results] + [r['doc_2'] for r in compare_results]))
all_choices = [(d, d) for d in docs]

@interact(
    seed=Dropdown(options=all_choices, value=all_choices[0][0], description='เลือกเอกสารเริ่มต้น'),
    round_ndigits=IntSlider(value=3, min=0, max=6, step=1, description='ทศนิยม'),
    sort_desc=Checkbox(value=True, description='เรียงจากมาก→น้อย'),
    also_print=Checkbox(value=False, description='พิมพ์บรรทัดสรุป')
)
def _view(seed, round_ndigits, sort_desc, also_print):
    # ดึงโหมดจาก PROCESSING_MODE ถ้ามี (จะทำให้ widget ใช้โหมดเดียวกับที่คำนวณผล)
    try:
        current_mode = PROCESSING_MODE
    except NameError:
        current_mode = 2  # ไม่มีตัวแปร กำหนดเป็นโหมดภาพ

    out = list_all_vs_seed(
        seed,
        round_ndigits=round_ndigits,
        sort_desc=sort_desc,
        also_print=also_print,
        mode=current_mode
    )

    # คอลัมน์จอแสดงผลให้เหมาะตามโหมด
    if current_mode == 1:
        cols = ['doc_1','doc_2','semantic','lexical','similarity_rounded','message']
    else:
        cols = ['doc_1','doc_2','similarity_rounded','matched_images','total_images','message']

    display(out[cols])



interactive(children=(Dropdown(description='เลือกเอกสารเริ่มต้น', options=(('663380118_0', '663380118_0'), ('6…

# Clustering


In [ ]:
!pip -q install networkx

# Deleter


In [188]:
# text_index.delete(delete_all=True)
image_index.delete(delete_all=True)

print("ลบข้อมูลใน pinecone ทั้งหมด")


ลบข้อมูลใน pinecone ทั้งหมด


In [189]:
import os
import glob

# ค้นหาไฟล์ .pdf ทั้งหมดใน /content
pdf_files = glob.glob('/content/*.pdf')

# ลบไฟล์ .pdf ทีละไฟล์
for file_path in pdf_files:
    try:
        os.remove(file_path)
        print(f'ลบไฟล์: {file_path}')
    except Exception as e:
        print(f'ไม่สามารถลบ {file_path} ได้: {e}')


ลบไฟล์: /content/663380114-8.pdf
ลบไฟล์: /content/663380108-3.pdf
ลบไฟล์: /content/663380140-7.pdf
ลบไฟล์: /content/663380448-9.pdf
ลบไฟล์: /content/663380479-8.pdf
ลบไฟล์: /content/663380102-5.pdf
ลบไฟล์: /content/663380092-2.pdf
ลบไฟล์: /content/663380093-0.pdf
ลบไฟล์: /content/663380446-4.pdf
ลบไฟล์: /content/663380130-5.pdf


# Threshold

In [32]:
import json

list_compare = list(compare_results)

print(json.dumps(list_compare, indent=4, ensure_ascii=False))


[
    {
        "doc_1": "663380118_0",
        "doc_2": "663380129_5",
        "semantic_similarity": 0.0,
        "lexical_similarity": 0.0,
        "image_similarity": 0.7943396920511164,
        "level": "คล้ายสูง",
        "text_message": "ไม่มีการประมวลผลข้อความ",
        "matched_images": 0,
        "total_images": 1
    },
    {
        "doc_1": "663380118_0",
        "doc_2": "663380138_4",
        "semantic_similarity": 0.0,
        "lexical_similarity": 0.0,
        "image_similarity": 0.7217007022229429,
        "level": "คล้ายระดับกลาง",
        "text_message": "ไม่มีการประมวลผลข้อความ",
        "matched_images": 0,
        "total_images": 1
    },
    {
        "doc_1": "663380118_0",
        "doc_2": "663380445_5",
        "semantic_similarity": 0.0,
        "lexical_similarity": 0.0,
        "image_similarity": 0.9345678080306422,
        "level": "เหมือนมาก",
        "text_message": "ไม่มีการประมวลผลข้อความ",
        "matched_images": 1,
        "total_images": 1
    }

In [33]:
image_sims = [item['image_similarity']
                for item in list_compare]
print(json.dumps(image_sims, indent=4
                 , ensure_ascii=False))


[
    0.7943396920511164,
    0.7217007022229429,
    0.9345678080306422,
    0.6802724339555252,
    0.9300332832955602,
    0.8586037735161764,
    0.7651563093932774,
    0.8161393916207789,
    0.7869838165089749,
    0.7339765087716048,
    0.8065683354060709,
    0.7332865448706752,
    0.6891530185393016,
    0.9224556093150145,
    0.6705710562765254
]


In [37]:

threshold = np.sum(image_sims) / 15
print("Threshold =", threshold)


Threshold = 0.789587218918279


In [35]:
threshold = np.mean(image_sims)
print("Threshold =", threshold)


Threshold = 0.789587218918279
